In [ ]:
# 初始化
import os
import csv
import geopandas as gpd
import pandas as pd
from pyproj import CRS
import matplotlib.pyplot as plt
from tqdm import tqdm

# draw_config
plt.rcParams["font.family"] = ["SimHei", "WenQuanYi Micro Hei", "Heiti TC"]


In [ ]:
# src_file = r"E:\gis\data\Poor_CommercialArea\countprocess\people_fishnet_600\people_fishnet_600.shp"
# gdf = gpd.read_file(src_file)

# K_score计算
# gdf['k_score'] = (gdf['COV'] - gdf['COV'].mean()) / gdf['COV'].std()
# gdf.to_file(r'data\test.shp', encoding="utf-8")

'''
检查数据
'''
def checkdata(p):
    ts = gpd.read_file(p, encoding='utf-8')
    print("字段列表:", ts.columns.tolist())
    print("数据量：", len(ts))
    print(ts[ts.columns.tolist()].head(10))

path = r"E:\gis\data\Poor_CommercialArea\04_出行平均时间\grid_location\data.shp"
checkdata(path)


In [ ]:
"""
为shp添加索引字段
"""

def add_idx(input, output, index='idx', start=1):
    gdf = gpd.read_file(input)
    if index in gdf.columns:
        print(f"【警告】: 字段 '{index}' 已存在")
    gdf[index] = range(start, start + len(gdf))
    gdf.to_file(output)
    print('添加成功')

p1 = r"E:\gis\data\Poor_CommercialArea\00_base_naturalblock\nl_L5.shp"
p2 = "data/output.shp"
add_idx(input=p1, output=p2, index='idx', start=1)
checkdata(p2)


In [ ]:
'''
合并两表
    左连接
'''

p1 = r"E:\gis\data\Poor_CommercialArea\04_出行平均时间\gird_location\data.shp"
p2 = r"E:\gis\data\Poor_CommercialArea\04_出行平均时间\c_influence\c_influence_sec.shp"
gdf1 = gpd.read_file(p1)
gdf2 = gpd.read_file(p2)
merged_gdf = gpd.GeoDataFrame.merge(
    gdf1, gdf2,
    on='TARGET_FID',
    how='left',
    suffixes=('_left', '_right')
)
merged_gdf = merged_gdf.drop(columns='geometry_right')
merged_gdf.to_file(r'data\data.shp')
checkdata(r'data\data.shp')


In [ ]:
'''
apply
'''

p1 = r"E:\CODE\Poor CommercialArea\data\data1.shp"
gdf = gpd.read_file(p1)

gdf['avetime'] = gdf['avetime'].apply(round)

gdf.to_file(r'data\data.shp')
checkdata(r'data\data.shp')


In [ ]:
'''
合并餐饮和购物poi连接到150渔网的连接点数量
'''

foodpoi = r"E:\gis\data\Poor_CommercialArea\countprocess\foodpoi_fishnet_150\foodpoi_fishnet_150.shp"
shoppoi = r"E:\gis\data\Poor_CommercialArea\countprocess\shoppoi_fishnet_150\shop_fishnet_150.shp"
fd = gpd.read_file(foodpoi)
sp = gpd.read_file(shoppoi)

fd['food_num'] = fd['Join_Count']
fd['shop_num'] = sp['Join_Count']
fd.to_file(r'data\foodshop_poi.shp', encoding="utf-8")

ts = gpd.read_file(r'data\foodshop_poi.shp')
col = ts.columns.tolist()
print("字段列表:", col)
print(ts[col].head(10))


In [ ]:
"""
空间权重分配,将600渔网人口数据按相交面积比例分配到自然街区
"""

def sp_allocation(fishnet_path, nl_path, output):
    grid = gpd.read_file(fishnet_path)
    nl = gpd.read_file(nl_path)

    if grid.crs != nl.crs:
        # 同步坐标系
        nl = nl.to_crs(grid.crs)
    if grid.crs.is_geographic:
        # 自动估算最佳UTM CRS
        utm_crs = grid.estimate_utm_crs()
        grid = grid.to_crs(utm_crs)
        nl = nl.to_crs(grid.crs)
    
    if 'cov' not in nl.columns:
        nl['cov'] = 0
        nl['ingird'] = 0
    for idx, grid_cell in tqdm(grid.iterrows(), total=len(grid)):
        grid_geom = grid_cell['geometry']
        intersect_nl = nl[nl.intersects(grid_geom)]

        if intersect_nl.empty:
            continue
        
        nlingird_area = 0
        for _, i in intersect_nl.iterrows(): 
            intersection = i['geometry'].intersection(grid_geom)
            nlingird_area += intersection.area
        for _, i in intersect_nl.iterrows(): 
            intersection = grid_geom.intersection(i['geometry'])
            intersection_area = intersection.area
            area_ratio = intersection_area / nlingird_area
            cov = grid_cell.get('COV', 0)
            allocat_cov = cov * area_ratio
            # log
            # print('id:',idx, '人数:',cov, '比例:',area_ratio, '分配人数:',allocat_cov, '相交面积:',intersection_area, '方格内街区总面积:',nlingird_area)
            nl.loc[nl['idx']==i['idx'], 'cov'] += allocat_cov
            nl.loc[nl['idx']==i['idx'], 'ingird'] += 1
    nl['cov'] = [round(i) for i in nl['cov']]
    nl.to_file(output)


p1 = r"E:\gis\data\Poor_CommercialArea\countprocess\nl_people_fishnetnet600\people_fishnet600\people_fishnet600.shp"
p2 = r"E:\gis\data\Poor_CommercialArea\00_base_naturalblock\nl_L5.shp"
oup = r"data/output.shp"
sp_allocation(p1, p2, oup)
checkdata(oup)


In [ ]:
'''
对应字段写入新字段
    56个方格写入对应的商圈名
'''

csv_path = r"E:\gis\data\Poor_CommercialArea\02_热门商业空间分布格局图\ame.csv"
shp_path = r"E:\gis\data\Poor_CommercialArea\02_热门商业空间分布格局图\first_select.shp"
output_path = r"E:\gis\data\Poor_CommercialArea\02_热门商业空间分布格局图\real\aaa.shp"

csv_data = pd.read_csv(csv_path)
gdf = gpd.read_file(shp_path)
# 创建匹配字典，键为user_quantity，值为name
quantity_to_name = dict(zip(csv_data['user_quant'], csv_data['name']))
gdf['bussiness'] = gdf['user_quant'].map(quantity_to_name).fillna('')
gdf.to_file(output_path, encoding='utf-8')


In [ ]:
'''
单字段数据粗去重
    对130w条数据的TARGET_FID去重
'''

shp_path = r"E:\gis\data\Poor_CommercialArea\countprocess\odsignl_fishnet600\od.shp"
gdf = gpd.read_file(shp_path)
print(f"原始记录数: {len(gdf)}")
gdf_unique = gdf.drop_duplicates(subset="TARGET_FID", keep="first")
output_path = "data.shp"
gdf_unique.to_file(output_path)
print(f"去重后记录数: {len(gdf_unique)}")
print(f"已删除 {len(gdf) - len(gdf_unique)} 条重复记录")

In [ ]:
'''
计算各方格前往各热门商业空间的人数
'''

od_path = r"E:\gis\data\Poor_CommercialArea\countprocess\odsignl_fishnet600\od.shp"
hotspot = r"E:\gis\data\Poor_CommercialArea\countprocess\Hot_Carea\Hot_Carea.shp"
target = r"E:\gis\data\Poor_CommercialArea\00_base_fishnet\base_fishnet600_id.shp"

od = gpd.read_file(od_path)
hp = gpd.read_file(hotspot)
tg = gpd.read_file(target)


hot_carea = {(row['d_lng'], row['d_lat']): row['bussiness'] for idx, row in hp.iterrows()}

od['bussiness'] = od.apply(
    lambda row: hot_carea.get((row['d_lng'], row['d_lat']), None),
    axis=1
)

od_agg = od.groupby(['TARGET_FID', 'bussiness'])['user_quant'].sum().reset_index()

agg_dict = od_agg.pivot_table(
    index='TARGET_FID', 
    columns='bussiness', 
    values='user_quant'
).fillna(0).to_dict('index')

for idx, row in tg.iterrows():
    fid = row['TARGET_FID']
    if fid in agg_dict:
        for buss, val in agg_dict[fid].items():
            tg.at[idx, buss] = val
    tg.at[idx, 'idx'] = fid

oup = 'data\output.shp'
tg.to_file(oup)
checkdata(oup)


In [ ]:
'''
计算各方格服务势力
    前往热门商业空间人数占全部前往热门商业空间人数占比最高的,且是否0.2。共两个字段
'''

cinfluence = r"E:\gis\data\Poor_CommercialArea\03_服务势力范围图\C_influence\c_influence.shp"
gdf = gpd.read_file(cinfluence)

rename_dict = {
    '农林下G': '农林下',
    '北京路-': '北京路',
    '大西关-': '大西关',
    '天河路-': '珠江新',
    '广州塔-': '广州塔',
    '金融城-': '金融城',
}
gdf = gdf.rename(columns=rename_dict)

fields_to_calculate = ['农林下', '北京路', '大西关', '珠江新', '奥体', '广州塔', '江南西', '环市东', '白鹅潭', '金融城', '广州北', '广州南']
gdf["total"] = gdf[fields_to_calculate].sum(axis=1)
gdf["maxvalue"] = gdf[fields_to_calculate].max(axis=1)
gdf["ratio"] = gdf.apply(
    lambda row: row["maxvalue"] / row["total"] if row["total"] != 0 else 0,
    axis=1
)
gdf["maxfield"] = gdf[fields_to_calculate].idxmax(axis=1)
name_mapping = {
    '农林下': '农林下路-中山三路',
    '北京路': '北京路-海珠广场商圈',
    '大西关': '大西关(上下九-永庆坊)商圈',
    '珠江新': '天河路-珠江新城商圈',
    '奥体': '奥体',
    '广州塔': '广州塔-琶洲商圈',
    '广州北': '广州大道北',
    '广州南': '广州大道南',
    '江南西': '江南西',
    '环市东': '环市东',
    '白鹅潭': '白鹅潭商圈',
    '金融城': '金融城-黄埔湾商圈',
}
gdf["maxfield"] = gdf["maxfield"].map(name_mapping).fillna(0)
gdf["ill"] = [1 if i>0.2 else 0 for i in gdf["ratio"]]

oup = "data\output.shp"
gdf.to_file(oup)
